<a href="https://colab.research.google.com/github/qldirr/pytorch_classification/blob/main/keras_car_crash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install icrawler

In [3]:
#사고 데이터 수집
# 데이터 크롤링을 사용해서 데이터셋을 만들고, 이미지를 학습하는 모델
from icrawler.builtin import GoogleImageCrawler
google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4,
                                   storage={'root_dir': './data'})

google_crawler.crawl(keyword = 'car crash', max_num=500,
                    min_size=(200,200), max_size=(None))

2022-02-15 04:15:35,690 - INFO - icrawler.crawler - start crawling...
2022-02-15 04:15:35,692 - INFO - icrawler.crawler - starting 1 feeder threads...
2022-02-15 04:15:35,706 - INFO - feeder - thread feeder-001 exit
2022-02-15 04:15:35,706 - INFO - icrawler.crawler - starting 2 parser threads...
2022-02-15 04:15:35,722 - INFO - icrawler.crawler - starting 4 downloader threads...
2022-02-15 04:15:36,288 - INFO - parser - parsing result page https://www.google.com/search?q=car+crash&ijn=1&start=100&tbs=&tbm=isch
2022-02-15 04:15:36,377 - INFO - parser - parsing result page https://www.google.com/search?q=car+crash&ijn=0&start=0&tbs=&tbm=isch
2022-02-15 04:15:36,512 - INFO - downloader - image #1	https://nypost.com/wp-content/uploads/sites/2/2022/02/020622accident03SG.jpg
2022-02-15 04:15:36,513 - INFO - downloader - image #2	https://ewscripps.brightspotcdn.com/ba/cc/462843b84649bf86684e92566f4b/chulavistafatal.jpg
2022-02-15 04:15:36,556 - INFO - downloader - image #3	https://nypost.com/

In [5]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split

# 분류 대상 카테고리 선택하기 
accident_dir = "./image"
categories = ["Car front crash","Car side crash","Rear and crash","Car broken windshield",
              "Car scratch","Flat tire","Overturned vehicle"]
nb_classes = len(categories)
# 이미지 크기 지정 
image_w = 64 
image_h = 64
pixels = image_w * image_h * 3
# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    # 이미지 
    image_dir = (r"/content/data")
    #image_dir = accident_dir + '/' + cat
    files = glob.glob(image_dir+"/*.jpg")
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
            
X = np.array(X)
Y = np.array(Y)
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)

print('>>> data 저장중 ...')
np.save("7obj.npy", xy)
print("ok,", len(Y))

0 
 [[[ 55  69  35]
  [ 53  64  34]
  [ 57  71  32]
  ...
  [ 61  61  60]
  [ 44  44  37]
  [ 78  72  68]]

 [[ 64  78  43]
  [ 58  69  38]
  [ 66  81  40]
  ...
  [ 31  33  27]
  [ 43  45  37]
  [ 94  88  80]]

 [[ 54  69  34]
  [ 51  62  33]
  [ 63  77  42]
  ...
  [ 26  29  22]
  [ 40  41  35]
  [ 88  83  75]]

 ...

 [[132 118 112]
  [129 113 108]
  [126 113 105]
  ...
  [197 176 173]
  [189 167 166]
  [184 161 164]]

 [[ 95  88  77]
  [ 88  81  64]
  [ 79  78  56]
  ...
  [195 171 168]
  [184 161 159]
  [184 160 159]]

 [[ 95  95  65]
  [ 90  88  59]
  [ 98  99  65]
  ...
  [195 171 167]
  [184 163 158]
  [182 159 157]]]
10 
 [[[ 34  35  40]
  [ 55  57  56]
  [ 67  67  67]
  ...
  [171 188 208]
  [171 188 208]
  [171 189 211]]

 [[ 23  26  30]
  [ 68  71  70]
  [ 75  75  77]
  ...
  [183 200 214]
  [181 198 213]
  [187 201 215]]

 [[ 22  22  26]
  [ 66  67  68]
  [ 87  89  90]
  ...
  [184 200 214]
  [190 204 216]
  [196 208 218]]

 ...

 [[143 134  99]
  [152 145 107]
  [157 148 

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [6]:
#모델 선언(CNN)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

# 카테고리 지정하기
categories = ["Car front crash","Car side crash","Rear and crash","Car broken windshield",
              "Car scratch","Flat tire","Overturned vehicle"]
nb_classes = len(categories)

# 이미지 크기 지정하기
image_w = 64
image_h = 64
# 데이터 열기 
X_train, X_test, y_train, y_test = np.load("./7obj.npy", allow_pickle=True)
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#전결합층
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

#모델 구축하기, 케라스의 경우 compile을 통해 구축한다.
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

#모델 확인
print(model.summary())

#학습이 완료된 모델 저장
hdf5_file = "./7obj-model.hdf5"
if os.path.exists(hdf5_file): 
    model.load_weights(hdf5_file)
    
else:
    model.fit(X_train, y_train, batch_size=32, epochs=100)
    model.save_weights(hdf5_file)


X_train shape: (341, 64, 64, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0         
                        

In [7]:
#모델의 오차 및 정확도 확인
score = model.evaluate(X_train,y_train)
print('loss=', score[0])
print('accuracy=', score[1])

11/11 [==============================] - 1s 91ms/step - loss: 1.7467 - accuracy: 0.1906
loss= 1.746748447418213
accuracy= 0.19061583280563354


In [9]:
#모델에 적용시키기

#적용시킬 이미지 선택
test_image = r'/content/data/000001.jpg'

img = Image.open(test_image)
img = img.convert('RGB')
img = img.resize((64,64))
data = np.asarray(img)
X = np.array(data)
X = X.astype('float') / 256
X = X.reshape(-1,64,64,3)

#예측
pred = model.predict(X)
result = [np.argmax(value) for value in pred]
print('Data Category is :', categories[result[0]])



Data Category is : Car scratch
